<a href="https://colab.research.google.com/github/romenlaw/llm_playground/blob/main/pdf_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 84.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
openai_key = userdata.get('openai_key')

In [ ]:
import fitz  # PyMuPDF
import openai

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Function to query an AI model with the extracted text
def ask_ai_about_pdf(pdf_text, question, api_key):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an AI that answers questions based on a provided document."},
            {"role": "user", "content": f"Document: {pdf_text}"},
            {"role": "user", "content": f"Question: {question}"}
        ],
        api_key=api_key
    )
    return response['choices'][0]['message']['content']

# Example usage
if __name__ == "__main__":
    pdf_path = "sample.pdf"  # Provide the path to your PDF file
    pdf_text = extract_text_from_pdf(pdf_path)

    api_key = "your-api-key-here"  # Replace with your OpenAI API key
    question = "What is the main topic of the document?"

    answer = ask_ai_about_pdf(pdf_text, question, api_key)
    print("Answer:", answer)

In [2]:
# Required libraries
import PyPDF2
import openai
import os
from time import sleep
from getpass import getpass

# Set up your AI API key (replace 'your-api-key' with your actual key)
openai.api_key = openai_key

def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.
    Args:
        pdf_path (str): Path to the PDF file
    Returns:
        str: Extracted text from the PDF
    """
    try:
        with open(pdf_path, 'rb') as file:
            pdf_reader = PyPDF2.PdfReader(file)
            text = ''
            for page in pdf_reader.pages:
                text += page.extract_text() + '\n'
        return text
    except Exception as e:
        print(f"Error reading PDF: {e}")
        return ""

def ask_ai_question(pdf_text, question):
    """
    Ask a question to the AI about the PDF content.
    Args:
        pdf_text (str): Text extracted from the PDF
        question (str): Question to ask the AI
    Returns:
        str: AI's response
    """
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Use the appropriate model
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the provided document text."},
                {"role": "user", "content": f"Document text: {pdf_text}\nQuestion: {question}"}
            ]
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        return f"Error communicating with AI: {e}"



Enter API key:··········


In [4]:
# Get the PDF path from user
pdf_path = './book5.pdf'

# Check if file exists
if not os.path.exists(pdf_path):
    print("File not found. Please check the path and try again.")

# Extract text from PDF
pdf_text = extract_text_from_pdf(pdf_path)
print("PDF text extracted successfully! You can now ask questions.")

In [5]:

while True:
    question = input("Ask a question about the PDF (or type 'quit' to exit): ").strip()
    if question.lower() == 'quit':
        print("Exiting...")
        break

    # Get answer from AI
    answer = ask_ai_question(pdf_text, question)
    print(f"\nQuestion: {question}")
    print(f"Answer: {answer}\n")
    input("Press Enter to ask another question...")


Ask a question about the PDF (or type 'quit' to exit): did John Snow survive in the end?

Question: did John Snow survive in the end?
Answer: Error communicating with AI: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


Press Enter to ask another question...quite
Ask a question about the PDF (or type 'quit' to exit): quit
Exiting...


# Hugging Face
Rubbish!

In [3]:
import fitz  # PyMuPDF for PDF reading
from transformers import pipeline  # Hugging Face for question-answering
import re

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    try:
        # Open the PDF
        doc = fitz.open(pdf_path)
        text = ""

        # Iterate through each page
        for page in doc:
            text += page.get_text("text")

        # Close the document
        doc.close()

        # Clean up text (remove extra spaces, newlines, etc.)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

# Function to answer questions based on extracted text
def answer_question(context, question):
    try:
        # Load a pre-trained question-answering model
        qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

        # Get the answer
        result = qa_pipeline(question=question, context=context)
        return result['answer']
    except Exception as e:
        return f"Error answering question: {str(e)}"



In [4]:
# Specify the PDF file path
pdf_path = "book5.pdf"  # Replace with your PDF file path

# Extract text from the PDF
context = extract_text_from_pdf(pdf_path)

In [9]:
len(context)

2308825

In [ ]:
# Main function to tie it all together
if not context.startswith("Error"):
    print("PDF text extracted successfully!")

    # Loop to ask questions
    while True:
        question = input("Ask a question about the PDF (or type 'exit' to quit): ")
        if question.lower() == 'exit':
            break

        # Get the answer
        answer = answer_question(context, question)
        print(f"Answer: {answer}\n")
else:
    print(context)

PDF text extracted successfully!
Ask a question about the PDF (or type 'exit' to quit): did John Snow survive in the end?


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu
